In [2]:
import pandas as pd


In [30]:
bus_route = pd.read_csv("../data/정류소_노선.csv")
bus_weight1 = pd.read_csv("../data/usage.csv")
total_data = pd.read_csv("../data/busstop.csv")

In [2]:
bus_weight = pd.DataFrame({"routeid1" :bus_weight1["routeid"], "weight" :bus_weight1["usage"]})

# routeid1 열을 explode하여 개별 routeid로 분리
exploded_bus_route = bus_route.explode('routeid1').reset_index(drop=True)

# bus_route와 bus_weight를 routeid1 기준으로 병합
merged_df = pd.merge(exploded_bus_route, bus_weight, on="routeid1", how="left")

# nodeid1별로 weight를 합산
nodeid1_weight = merged_df.groupby('nodeid1')['weight'].sum().reset_index()


## 기존 총데이터에 가중치 추가

total_data = pd.merge(total_data, nodeid1_weight, on='nodeid1', how='left') 





# 결과를 bus_route에 병합하여 가중치 업데이트
# 먼저 bus_route를 explode하여 각 nodeid1에 대한 weight를 병합
#bus_route_exploded = bus_route.explode('routeid1').reset_index(drop=True)
#bus_route_with_weight = pd.merge(bus_route_exploded, bus_weight, on='routeid1', how='left')

# 원래 형태로 복구하면서 nodeid1별 weight를 합산
#final_result = bus_route_with_weight.groupby(['rigeon1', 'nodeid1']).agg({
#    'routeid1': lambda x: list(x),
#    'weight': 'sum'
#}).reset_index()


In [ ]:
total_data